<a href="https://colab.research.google.com/github/before-born/Neuron/blob/main/lstmbasedautoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the IMDB dataset
max_features = 10000  # Limit the number of words in the vocabulary
max_len = 100  # Maximum length of the input sequences

(x_train, _), (x_test, _) = imdb.load_data(num_words=max_features)

# Pad sequences to ensure uniform input size
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Define the LSTM Autoencoder model
timesteps = max_len
input_dim = max_features

# Encoder
encoder_inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(64, activation='relu', return_sequences=False)(encoder_inputs)
latent_dim = 32
encoded_latent = Dense(latent_dim, activation='relu')(encoded)

# Latent Space Representation
latent_inputs = Input(shape=(latent_dim,))
decoded_latent = Dense(64, activation='relu')(latent_inputs)
decoded = RepeatVector(timesteps)(decoded_latent)
decoded = LSTM(input_dim, activation='sigmoid', return_sequences=True)(decoded)

# Define the full Autoencoder model
autoencoder = Model(encoder_inputs, decoded)

# Define the Encoder model
encoder_model = Model(encoder_inputs, encoded_latent)

# Define the Decoder model
decoder_inputs = Input(shape=(latent_dim,))
decoder_outputs = autoencoder.layers[-2](decoder_inputs)
decoder_outputs = autoencoder.layers[-1](decoder_outputs)
decoder_model = Model(decoder_inputs, decoder_outputs)

# Compile the Autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
autoencoder.fit(x_train, x_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model
loss = autoencoder.evaluate(x_test, x_test)
print(f"Test loss: {loss:.4f}")
